<img align="left" style="padding-right:10px;" src="figures/cartel.jpg">
<!--COURSE_INFORMATION-->
## This notebook contains the index from the course [Biology Meets Programming](https://www.coursera.org/learn/bioinformatics/home/welcome) by University of California in Coursera 


### The content is available [on GitHub](https://github.com/vencejo/Curso_BiologyMeetsProgramming).

<!--NAVIGATION-->
< [3.3 Scoring Motifs](3.3 Scoring Motifs.ipynb)| [Contents](Index.ipynb) | [3.5 Detour Discovery of Codons and Split Genes](3.5 Detour Discovery of Codons and Split Genes.ipynb)>


### Using the profile matrix to roll dice

Many algorithms are iterative procedures that must choose among various alternatives at each iteration. Some of these alternatives may lead to correct solutions, whereas others may not. ** Greedy algorithms ** select the “most attractive” alternative at each iteration. For example, a greedy algorithm in chess might attempt to capture an opponent’s most valuable piece at every move. Yet anyone who has played chess knows that a strategy looking only one move ahead will likely produce disastrous results.

In general, most greedy algorithms typically fail to find an exact solution of the problem; instead, they are often fast heuristics that trade accuracy for speed in order to find an approximate solution. Nevertheless, for many biological problems, greedy algorithms will prove quite useful.

In this section, we will explore a greedy approach to motif finding. Again, let Motifs be a collection of k-mers taken from t strings Dna. We can view each column of Profile(Motifs) as a four-sided biased die with one nucleotide on each side. Thus, a profile matrix with k columns can be viewed as a collection of k dice that we will roll to randomly generate a k-mer. For example, if the first column of the profile matrix is (0.2, 0.1, 0.0, 0.7), then we generate A as the first nucleotide with probability 0.2, C with probability 0.1, G with probability 0.0, and T with probability 0.7.

<img align="center" style="padding-right:10px;" src="figures/fig42.png">


<img align="center" style="padding-right:10px;" src="figures/fig43.png">



To implement a function Pr(Text, Profile), we begin by setting a “probability” variable p equal to 1. We then range through the characters of Text one at a time. At position i of Text, we set p equal to p times the value of Profile corresponding to symbol Text(i) and column i, which is just Profile(Text(i]))[i].

Code Challenge (2 points): Use this idea to write a function Pr(Text, Profile) and add it to Motifs.py.

Click here to download this problem's test datasets.

Sample Input:

ACGGGGATTACC
0.2 0.2 0.0 0.0 0.0 0.0 0.9 0.1 0.1 0.1 0.3 0.0
0.1 0.6 0.0 0.0 0.0 0.0 0.0 0.4 0.1 0.2 0.4 0.6
0.0 0.0 1.0 1.0 0.9 0.9 0.1 0.0 0.0 0.0 0.0 0.0
0.7 0.2 0.0 0.0 0.1 0.1 0.0 0.5 0.8 0.7 0.3 0.4

Sample Output:

0.0008398080000000002


```python
# Input:  String Text and profile matrix Profile
# Output: Pr(Text, Profile)
def Pr(Text, Profile):
    p = 1
    for i in range(len(Text)):
        p *= Profile[Text[i]][i]
    return p
```





Given a profile matrix Profile, we can compute the probability of every k-mer in a string Text and find a Profile-most probable k-mer in Text, i.e., a k-mer that was most likely to have been generated by Profile among all k-mers in Text. For the NF-κB profile matrix, "ACGGGGATTACC" is the Profile-most probable 12-mer in "ggtACGGGGATTACCt". Indeed, every other 12-mer in this string has probability 0. In general, if there are multiple Profile-most probable k-mers in Text, then we select the first such k-mer occurring in Text.

Profile-most Probable k-mer Problem: Find a Profile-most probable k-mer in a string.

* Input: A string Text, an integer k, and a 4 x k matrix Profile.
* Output: A Profile-most probable k-mer in Text.

Code Challenge (3 points): Solve the Profile-most Probable k-mer Problem by writing a function ProfileMostProbablePattern(Text, k, Profile). (Hint: make sure to use the function Pr(Text, Profile) as a subroutine.)

Click here to download this problem's test datasets.

Sample Input:

ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT
5
0.2 0.2 0.3 0.2 0.3
0.4 0.3 0.1 0.5 0.1
0.3 0.3 0.5 0.2 0.4
0.1 0.2 0.1 0.1 0.2

Sample Output:

CCGAG

```python
def Pr(Text, Profile):
    p = 1
    for i in range(len(Text)):
        p *= Profile[Text[i]][i]
    return p

def ProfileMostProbablePattern(Text, k, Profile):
    """ Explicación de porque poner la pMax a -1
    (Sacado del comentario de Cris Lawrence ) a la tarea:
    I too did this in PyCharm and got the expected answer but failed the test until
    I also set p to -1 initially as someone else did.
    I guess the issue is that by setting p = 0 for starters,
    you could wind up with just an empty string if all possibilities
    occur with 0 probability.  The algorithm spec says return the first best.
    If all have 0 probability, that would be the first 5-mer in the sequence.
    Setting the probability to less than 0 guarantees you return something.
    I guess that's why an initial value for p of 0 doesn't work. """
    pMax = -1
    mostProbPattern = ""
    for i in range(0, len(Text)-k):
        pattern = Text[i:i+k]
        p = Pr(pattern, Profile )
        if p > pMax:
            mostProbPattern = pattern
            pMax = p

    return mostProbPattern

```



Our proposed greedy motif search algorithm, GreedyMotifSearch, starts by setting BestMotifs equal to the first k-mer from each string in Dna. These strings will serve as the best-scoring motifs found thus far.

```python
    BestMotifs = []
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k])

```

It then ranges over all possible k-mers in Dna[0], trying each one as Motifs[0]. For a given choice of k-mer in Dna[0] for Motifs[0], the algorithm then builds a profile matrix Profile for this lone k-mer, and sets Motifs[1] equal to the Profile-most probable k-mer in Dna[1]. GreedyMotifSearch then iterates by updating Profile as the profile matrix formed from Motifs[0] and Motifs[1], and sets Motifs[2] equal to the Profile-most probable k-mer in Dna[2]. In general, after finding k-mers Motifs in the first i strings of Dna, GreedyMotifSearch constructs Profile(Motifs) and sets Motifs[i] equal to the Profile-most probable k-mer from Dna[i] based on this profile matrix.

```python
    n = len(Dna[0])
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = Profile(Motifs[0:j])
            Motifs.append(ProfileMostProbablePattern(Dna[j], k, P))

```

After selecting a k-mer from each string in Dna to obtain a collection of strings Motifs, GreedyMotifSearch checks whether Motifs outscores the current best scoring collection of motifs, BestMotifs.

```python

if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs

```

It then returns to the top of the for loop and moves one symbol over in Dna[0], beginning the entire process of generating  Motifs again. After generating a collection Motifs for every possible initial k-mer from Dna[0], it returns the high-scoring strings BestMotifs.

Note: Graeme Benstead-Hume wrote a [post](http://www.mrgraeme.co.uk/greedy-motif-search/) explaining GreedyMotifSearch in greater detail. Graeme was one of the excellent learners in our first ever MOOC session, and has since gone on to become a PhD student in bioinformatics at the University of Sussex! If you need more time to understand the algorithm, you may want to check out his post.

Code Challenge (1 point): Consolidate this code into a function GreedyMotifSearch(Dna, k, t), and then add this function to Motifs.py.

Click here to download this problem's test datasets.

Sample Input:

3 5
GGCGTTCAGGCA
AAGAATCAGTCA
CAAGGAGTTCGC
CACGTCAATCAC
CAATAATATTCG

Sample Output:

CAG
CAG
CAA
CAA
CAA

```python
def GreedyMotifSearch(Dna, k, t):
    BestMotifs = []
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k])

    n = len(Dna[0])
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = Profile(Motifs[0:j])
            Motifs.append(ProfileMostProbablePattern(Dna[j], k, P))
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs

    return BestMotifs


```


### Motifs in tuberculosis

Tuberculosis (TB) is an infectious disease that is caused by the Mycobacterium tuberculosis bacterium (MTB) and is responsible for over a million deaths each year. Although the spread of TB has been greatly reduced due to antibiotics, strains that resist all available treatments are now emerging. MTB is successful as a pathogen because it can persist in humans for decades without causing disease; in fact, one-third of the world population has latent MTB infections, in which MTB lies dormant within the host’s body and may or may not reactivate at a later time. The widespread prevalence of latent infections makes it difficult to control TB epidemics. Biologists are therefore interested in finding out what makes the disease latent and how MTB activates itself within a host.

It remains unclear why MTB can stay latent for so long and how it survives during latency. The resistance of latent TB to antibiotics implies that MTB may have an ability to shut down expression of most genes and stay dormant, not unlike bears hibernating in the winter. Hibernation in bacteria is called sporulation because many bacteria form protective and metabolically dormant spores that can survive in tough conditions, allowing the bacteria to persist in the environment until conditions improve.

Hypoxia, or oxygen shortage, is often associated with latent forms of TB. Biologists have found that MTB becomes dormant in low-oxygen environments, presumably with the idea that the host’s lungs will recover enough to potentially spread the disease in the future. Since MTB shows a remarkable ability to survive for years without oxygen, it is important to identify MTB genes responsible for the development of the latent state under hypoxic conditions. Biologists are interested in finding a master regulator (transcription factor) that “senses” the shortage of oxygen and starts a genetic program that affects the expression of many genes, allowing MTB to adapt to hypoxia.



In 2003, biologists found the dormancy survival regulator (DosR), a transcription factor that regulates many genes whose expression dramatically changes under hypoxic conditions. However, it remained unclear how DosR regulates these genes, and its transcription factor binding site remained unknown. In an attempt to resolve this puzzle, biologists performed a DNA array experiment and found 25 genes whose expression levels significantly changed in hypoxic conditions. Given the upstream regions of these genes, each of which is 250 nucleotides long, we would like to discover the “hidden message” that DosR uses to control the expression of these genes.

To simplify the problem a bit, we have selected just 10 of the 25 genes, resulting in the DosR dataset (click here to download). In this chapter, we will try to identify motifs in this dataset using the motif finding algorithms that we will develop. However, we will not give you a hint about the DosR motif.



Exercise Break (1 point): Use GreedyMotifSearch to look for motifs in the DosR dataset with k-mer length equal to 15 (click [here](dnas/DosR.txt) to download).

Passed test #2. Correct! Below is the best set of Motifs in the DosR with k = 15 using GreedyMotifSearch (with a score of 64):
GTTAGGGCCGGAAGT





## Analyzing greedy motif finding

GreedyMotifSearch is fast and can be run with k = 15 to find candidate motifs in the DosR dataset, as we saw on the previous step.  At the same time, it trades speed for accuracy; when we run it on the Subtle Motif Dataset, GreedyMotifSearch returns the 15-mer "gtAAAtAgaGatGtG" (total score: 58), which varies greatly from the true implanted motif "AAAAAAAAGGGGGGG".  This makes us highly suspicious of the results we obtained when running this algorithm on the DosR dataset.

STOP and Think: Why does GreedyMotifSearch perform so poorly?

At first glance, GreedyMotifSearch may seem like a reasonable algorithm, but it is not! Let’s see whether GreedyMotifSearch will find the (4, 1)-motif "ACGT" implanted in the following strings Dna:

<img align="center" style="padding-right:10px;" src="figures/fig44.png">


The algorithm is now ready to search for a Profile-most probable 4-mer in the second string. The issue, however, is that there are so many zeroes in the profile matrix that the probability of every 4-mer but "ACCT" is zero! Thus, unless "ACCT" is present in every string in Dna, there is little chance that GreedyMotifSearch will find the implanted motif. Zeroes in the profile matrix are not just a minor annoyance but rather a persistent problem that we must address.

<!--NAVIGATION-->
< [3.3 Scoring Motifs](3.3 Scoring Motifs.ipynb)| [Contents](Index.ipynb) | [3.5 Detour Discovery of Codons and Split Genes](3.5 Detour Discovery of Codons and Split Genes.ipynb)>